<a href="https://colab.research.google.com/github/ElielLucas/Digital-Image-Processing/blob/main/AED3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image, ImageFont, ImageDraw, ImageFilter
from PIL.ImageChops import add, subtract, multiply, difference, screen
import PIL.ImageStat as stat
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
from skimage import color, viewer, exposure, img_as_float, data
from skimage.transform import SimilarityTransform, warp, swirl
from skimage.util import invert, random_noise, montage
import matplotlib.image as mpimage
import matplotlib.pylab as plt
from scipy.ndimage import affine_transform, zoom
from scipy import misc
import cv2
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
import os
from natsort import natsorted
import skimage
import time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Viewer requires Qt
  


In [ ]:
def extrair_frames(dir):
  vidcap = cv2.VideoCapture(dir)
  count = 0

  success,image = vidcap.read()
  while vidcap.isOpened() and success:
    cv2.imwrite("frames/frame%d.jpg" % count, image)
    count += 1
    success,image = vidcap.read()

  vidcap.release()
  cv2.destroyAllWindows()

In [ ]:
def list_frames(dir):
  pasta = os.listdir(dir)
  pasta = natsorted(pasta)

  list_frames = []
  for frame in pasta:
    list_frames.append(cv2.imread(dir + frame))
  return list_frames

In [ ]:
def list_col(image, col, N):
  output = []
  for i in range(0, N):
      output.append(image[i][col])

  return output

In [ ]:
def contabilizar_pixels_imagem_anexo(frame):
    # resize frame
    dim = (720, 1280)
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)

    N, M, _ = frame.shape

    output = 0
    for i in range(N):
        for j in range(M):
            if (frame[i, j, 0] == 255):
                output += 1
    return output

In [ ]:
def get_frames_validos():
    # Lista com todos frames de máscara do vídeo
    mascara_frm = list_frames('/content/mascara/')

    lista_de_frames_validos = []
    it = 1
    # Itera sobre os frames da mascara e a cada 20 frames captura 1 válido
    for i in range(len(mascara_frm)):
        if it == 20:
           lista_de_frames_validos.append(mascara_frm[i])
           it = 0
        it += 1
    return lista_de_frames_validos

In [ ]:
def get_quantidade_adubo_video(mascara_adubo_anexo):
    # Conto quantos pixels brancos a imagem deixada em anexo, pelo monitor, tem.
    qtd_pixels_frame_anexo = contabilizar_pixels_imagem_anexo(mascara_adubo_anexo)
    print('Quantidade de pixels do frame em anexo: ', qtd_pixels_frame_anexo)
    adubo_total = 0
    # Itero por cada máscara do vídeo e estimo para cada um qual o seu volume de adubo, por meio de uma regra de 3.
    for i in range(len(lista_frames_validos)):
        # Conto quantos pixels a máscara atual tem
        qtd_pixels_frame_at = contabilizar_pixels_imagem_anexo(lista_frames_validos[i])
        # Faço uma regra de 3....Se X pixels de adubo da imagem em anexo equivalem a 1.38kg, então Y pixels de adubo da máscara equivalem a quanto?
        adubo_frame_at = (qtd_pixels_frame_at*1.38) / qtd_pixels_frame_anexo
        adubo_total += adubo_frame_at
        print('Frame válido: ', i + 1)
        print(f'Quantidade de pixels do frame: {qtd_pixels_frame_at}')
        print(f'Quantidade de adubo do frame: {adubo_frame_at}')

    return adubo_total

In [ ]:
def segmentar_frame_anexado():
    mn, mx = np.array([1, 50, 100]), np.array([14, 255, 215])
    mn2, mx2 = np.array([100, 40, 1]), np.array([175, 225, 75])
    im = cv2.imread("/content/adubo_kg.png")

    N, M, _ = im.shape
    img_yuv1 = cv2.cvtColor(im, cv2.COLOR_BGR2YUV)
    img_yuv1[:,:,0] = cv2.equalizeHist(img_yuv1[:,:,0])
    bilateral = cv2.bilateralFilter(img_yuv1,d=5,sigmaColor=5,sigmaSpace=35)

    im2 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im2 = cv2.GaussianBlur(im2,(7,7),cv2.BORDER_DEFAULT)
    t_lower = 20
    t_upper = 160
    edge = cv2.Canny(im2, t_lower, t_upper)

    mask1 = cv2.inRange(bilateral, mn, mx)
    mask2 = cv2.inRange(bilateral, mn2, mx2)
    mask = mask1 + mask2
    mask = edge + mask

    mask_dil = cv2.dilate(mask, np.ones((3, 3), dtype='uint8'), iterations=8)
    mask_ero = cv2.erode(mask_dil, np.ones((7, 7), dtype='uint8'), iterations=4)
    mask_dil = cv2.dilate(mask_ero, np.ones((3, 3), dtype='uint8'), iterations=11)

    for j in range(M):
      if j <= 110 or j >= 550:
        for i in range(N):
            mask_dil[i, j] = 0

    imshow(mask_dil), show()
    final_mask = cv2.cvtColor(mask_dil, cv2.COLOR_GRAY2BGR)

    return final_mask

In [ ]:
extrair_frames('video_adubo.mp4')
list_frm = []
list_frm = list_frames('/content/frames/')
qnt_frames = len(list_frm)
count = 0
for frame in range(0, qnt_frames - 1):
    im1 = list_frm[frame]
    im2 = list_frm[frame + 1]

    N, M, _ = im1.shape
    img_yuv1 = cv2.cvtColor(im1, cv2.COLOR_BGR2YUV)
    img_yuv1[:,:,0] = cv2.equalizeHist(img_yuv1[:,:,0])

    img_yuv2 = cv2.cvtColor(im2, cv2.COLOR_BGR2YUV)
    img_yuv2[:,:,0] = cv2.equalizeHist(img_yuv2[:,:,0])

    R, G, B = img_yuv1[:,:,0], img_yuv1[:,:,1], img_yuv1[:,:,2]
    im1_gray = 0.2989 * R + 0.5870 * G + 0.1140 * B
    R, G, B = img_yuv2[:,:,0], img_yuv2[:,:,1], img_yuv2[:,:,2]
    im2_gray = 0.2989 * R + 0.5870 * G + 0.1140 * B

    mascara = np.zeros((N, M, 3), dtype='uint8')
    for i in range(N):
        for j in range(M):
          dif = abs(int(im1_gray[i][j]) - int(im2_gray[i][j]))
          if (dif > 0.04 * int(im1_gray[i][j])):
              mascara[i, j, :] = 255

    mascara2 = np.zeros((N, M, 3), dtype='uint8')
    for j in range(M):
        val_rows = list_col(mascara, j, N)
        median = np.median(val_rows)
        if median:
          for i in range(N):
            mascara2[i, j, :] = median

    cv2.imwrite('mascara/frame%d.jpg' %count, mascara2)
    count += 1


lista_frames_validos = get_frames_validos()
anexo_segmentado = segmentar_frame_anexado()
adubo_total = get_quantidade_adubo_video(anexo_segmentado)
print("\n\n")
print(f'Quantidade total de adubo no vídeo: {adubo_total} kg')
